In [4]:
#觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，
#對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?
library(dplyr)
library(magrittr)
library(caret)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\house.train.csv',sep=',',header=T)
test=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\house.test.csv',sep=',',header=T)

##extract categorical variable
col.type=rep(0,81)
for (i in 1:81) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')


##only keep categorical column
train.cate=train[,c(col.factor,81)]
test.cate=test[,col.factor]

## covert NA into a level
for (j in 1:43) {
  train.cate[,j] %<>% addNA(.)
  
}


## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:43) {
  train.cate.label[,k] %<>% as.numeric(.)
}
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(SalePrice~., data=train.cate.label, trControl=train_control, method="lm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
rmse1=cv1$results[2]
rsq1=cv1$results[3]
fit1=lm(SalePrice~.,data=train.cate.label)
summary(fit1)



## linear regression + one hot coding

train_control <- trainControl(method="cv", number=5)
t11=Sys.time()
cv2= train(SalePrice~., data=train.cate, trControl=train_control, method="lm")
t12=Sys.time()

time2=difftime(t11,t12,units='secs')
rmse2=cv2$results[2]
rsq2=cv2$results[3]
fit2=lm(SalePrice~.,data=train.cate)
summary(fit2)


library(xgboost)
library(dummies)

## gboost + lable coding
t11=Sys.time()
xgb.fit1 <- xgb.cv(
  data = as.matrix(train.cate.label[,-44]),
  label = train.cate.label[,44],
  nrounds = 100,
  nfold = 5,
  objective = "reg:linear",  # for regression models
  verbose = 0               # silent,不要顯示詳細資訊
)
t12=Sys.time()
gb.mse1=mean(unlist(xgb.fit1$evaluation_log[,4]))
time.gb1=difftime(t11,t12,units='secs')

## gboost + one hot coding
train.dummy=train.cate[,44]
for (p in 1:43) {
  train.dummy=cbind(train.dummy,dummy(train.cate[,p]))
}

t11=Sys.time()
xgb.fit2 <- xgb.cv(
  data = as.matrix(train.dummy[,-1]),
  label = train.dummy[,1],
  nrounds = 100,
  nfold = 5,
  objective = "reg:linear",  # for regression models
  verbose = 0               # silent,不要顯示詳細資訊
)

t12=Sys.time()
gb.mse2=mean(unlist(xgb.fit2$evaluation_log[,4]))
time.gb2=difftime(t11,t12,units='secs')

######## 解析

regression.mse.diff=rmse2-rmse1
regression.mse.diff
regression.rsq.diff=rsq2-rsq1
regression.rsq.diff
xgb.mse.diff=gb.mse2-gb.mse1
xgb.mse.diff

regression.time.diff=time2-time1
regression.time.diff
xgb.time.diff=time.gb2-time.gb1
xgb.time.diff

Warning message:
"package 'caret' was built under R version 3.5.3"

ERROR: Error: package or namespace load failed for 'caret' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'codetools'


## linear regression and xgboost has better performance when we carry out  one hot coding. However, it is obvious that we need to spend much time calculating cv results, since one hot coding increase a lot of dummy variables.

In [6]:
library(dplyr)
library(magrittr)
library(caret)
#鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
#鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')



##only keep categorical column
train.cate=train[,c(col.factor,1)]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:3) {
  train.cate.label[,k] %<>% as.numeric(.)
}


t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(as.factor(Survived)~., data=train.cate.label, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
racc1=cv1$results[2]
rkappa1=cv1$results[3]


## linear regression + one hot coding
## gboost + one hot coding


train_control <- trainControl(method="cv", number=5)
t11=Sys.time()
cv2= train(as.factor(Survived)~., data=train.cate, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()

time2=difftime(t11,t12,units='secs')
racc2=cv2$results[2]
rkappa2=cv2$results[3]
summary(fit2)


library(xgboost)
library(dummies)

## gboost + lable coding
t11=Sys.time()
xgb.fit1 <- xgb.cv(
  data = as.matrix(train.cate.label[,-4]),
  label = train.cate.label[,4],
  nrounds = 100,
  nfold = 5,
  objective = "reg:logistic",  # for regression models
  verbose = 0               # silent,不要顯示詳細資訊
)
t12=Sys.time()
gb.mse1=mean(unlist(xgb.fit1$evaluation_log[,4]))
time.gb1=difftime(t11,t12,units='secs')


## gboost + one hot coding
train.dummy=train.cate[,4]
for (p in 1:3) {
  train.dummy=cbind(train.dummy,dummy(train.cate[,p]))
}

t11=Sys.time()
xgb.fit2 <- xgb.cv(
  data = as.matrix(train.dummy[,-1]),
  label = train.dummy[,1],
  nrounds = 100,
  nfold = 5,
  objective = "reg:logistic",  # for regression models
  verbose = 0               # silent,不要顯示詳細資訊
)

t12=Sys.time()
gb.mse2=mean(unlist(xgb.fit2$evaluation_log[,4]))
time.gb2=difftime(t11,t12,units='secs')

######## 解析

regression.acc.diff=racc2-racc1
regression.acc.diff
regression.kappa.diff=rkappa2-rkappa1
regression.kappa.diff
xgb.mse.diff=gb.mse2-gb.mse1
xgb.mse.diff

regression.time.diff=time2-time1
regression.time.diff
xgb.time.diff=time.gb2-time.gb1
xgb.time.diff

Warning message:
"package 'caret' was built under R version 3.5.3"

ERROR: Error: package or namespace load failed for 'caret' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'codetools'


## In titanic data which is a calssification problem,  logistic and xgboost still have better performance in one hot coding, and still, one hot coding is more time consumming than label coding.